# API KEY

In [1]:
# 파일을 읽기 모드로 열기
with open("C:\\Users\\lee\\Desktop\\bybit\\api.txt", "r") as file:
    f = []
    # 파일의 각 줄에 대해 반복
    for line in file:
        # 각 줄 출력
        f.append(line.strip())  # strip()은 줄 바꿈 문자(\n) 등을 제거하여 출력합니다.

# 캔들 마크 가격

In [12]:
from pybit.unified_trading import HTTP
from datetime import datetime
import math

now = datetime.now()
now_timestamp = math.floor(now.timestamp()*1000)

# get_kline / get_mark_price_kline
session = HTTP(testnet=True)
price = session.get_mark_price_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=1,
    start=now_timestamp-60000000,
    end=now_timestamp,
    limit=1000,
)

price['result']['list'][0:3]


[['1700651220000', '36603.37', '36603.59', '36600.7', '36600.7'],
 ['1700651160000', '36624.1', '36624.1', '36603.37', '36603.37'],
 ['1700651100000', '36623.5', '36627.7', '36621.1', '36624.1']]

In [3]:
a_hundred_prices = price['result']['list']
highPrice = []
lowPrice = []
closePrice = []
for i in range(len(a_hundred_prices)):
    # a = float(a_hundred_prices[i][2])
    # b = float(a_hundred_prices[i][3])
    # c = float(a_hundred_prices[i][4])
    # highPrice.append(a)
    # lowPrice.append(b)
    # closePrice.append(c)

    highPrice.append(float(a_hundred_prices[i][2]))
    lowPrice.append(float(a_hundred_prices[i][3]))
    closePrice.append(float(a_hundred_prices[i][4]))

# 가장 최신 가격을 앞에서 뒤로 변경하기 위한 코드
highPrice_reverse = highPrice[::-1]
lowPrice_reverse = lowPrice[::-1]
closePrice_reverse = closePrice[::-1]

# EMA

In [4]:
import talib
import numpy as np

price_data = np.array(closePrice_reverse)

ema_period = 50
ema_values = talib.EMA(price_data, timeperiod=ema_period)

ema_values


array([           nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan, 36888.3       , 36890.25647059, 36891.92484429,
       36894.90818373, 36901.80315692, 36907.24969978, 36913.85128018,
      

# RSI

In [5]:
import talib
import numpy as np

price_data = np.array(closePrice_reverse)

rsi_period = 14
rsi_values = talib.RSI(price_data, timeperiod=rsi_period)

rsi_values


array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan, 53.58260206,
       51.70048763, 45.3973827 , 47.18841147, 51.01574831, 48.33951409,
       42.92956753, 48.55808185, 50.5287159 , 45.70639238, 39.04431677,
       46.7980834 , 49.88448167, 53.60791811, 50.29032372, 44.55386049,
       44.62881222, 43.32022157, 47.62428618, 52.0618228 , 49.85349205,
       52.91064084, 47.51645458, 43.87056533, 41.67166751, 45.91993587,
       43.18471575, 41.71983245, 38.81079526, 35.82835898, 38.33422433,
       36.46038759, 39.5206803 , 46.83825969, 53.49467065, 56.44713417,
       63.31931376, 62.12860437, 66.55220268, 75.53405148, 69.64462325,
       72.34267745, 66.43540631, 71.19252359, 64.51999029, 66.14845768,
       66.75363034, 66.06844877, 66.49671594, 56.2403326 , 53.3967358 ,
       51.89643668, 49.16656475, 51.10825104, 51.50869699, 53.48

# ADX

In [6]:
import talib
import numpy as np

high_data = np.array(highPrice_reverse)
low_data = np.array(lowPrice_reverse)
close_data = np.array(closePrice_reverse)

adx_period = 14
adx_values = talib.ADX(high_data, low_data, close_data, timeperiod=adx_period)

adx_values

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  8.20645409,  7.85565111,  7.96198534,
        8.13424771,  8.44149856,  8.56555575,  8.09617477,  7.83202535,
        7.88966891,  7.69771706,  7.98400054,  8.58440035,  9.33989917,
       10.32195359, 11.34658772, 12.40343672, 13.9697008 , 15.93111581,
       17.9426805 , 19.81056199, 19.93329871, 18.6374241 , 17.82089242,
       18.30954926, 18.76330203, 19.8411    , 22.45880257, 24.88952637,
       27.25600751, 28.96676104, 30.92346572, 32.74040578, 34.41590019,
       36.23916749, 37.8997532 , 39.09223797, 38.28054496, 36.96375078,
       34.8896095 , 32.96362117, 31.04890018, 29.09453014, 27.66

# 실시간 호가

In [3]:
from pybit.unified_trading import WebSocket
from time import sleep
from datetime import datetime


ws = WebSocket(
    testnet=True,
    channel_type="linear",
)
def handle_message(message):
    close_price = message['data'][0]['close']
    time = datetime.utcfromtimestamp(message['ts']/1000)
    print(close_price, time)
    # print(message)
ws.kline_stream(
    interval=1,
    symbol="BTCUSDT",
    callback=handle_message
)
while True:
    sleep(1)

36656.8 2023-11-22 11:02:43.580000
36662.7 2023-11-22 11:02:47.578000
36662.7 2023-11-22 11:02:48.578000
36661.5 2023-11-22 11:02:50.578000
36661.5 2023-11-22 11:02:52.581000
36660.4 2023-11-22 11:02:53.580000
36661.6 2023-11-22 11:02:54.581000
36660.5 2023-11-22 11:02:55.578000
36660.5 2023-11-22 11:02:56.581000
36660.5 2023-11-22 11:02:57.578000
36660.4 2023-11-22 11:02:59.579000
36660.4 2023-11-22 11:03:00.578000
36660.5 2023-11-22 11:03:01.578000
36659 2023-11-22 11:03:02.579000
36659 2023-11-22 11:03:03.579000
36658.9 2023-11-22 11:03:04.578000
36662 2023-11-22 11:03:05.578000
36662.7 2023-11-22 11:03:06.580000
36663.3 2023-11-22 11:03:10.578000
36663.2 2023-11-22 11:03:11.578000
36663.9 2023-11-22 11:03:12.577000
36663.2 2023-11-22 11:03:14.578000
36661.3 2023-11-22 11:03:16.578000
36656.5 2023-11-22 11:03:17.579000
36656.5 2023-11-22 11:03:18.578000
36656.5 2023-11-22 11:03:20.580000
36663.5 2023-11-22 11:03:24.578000
36663.5 2023-11-22 11:03:25.580000
36663.5 2023-11-22 11:03:2